In [1]:
from PIL import Image 
import numpy as np
import os

f = open("infarcts.csv", "r")
naming_dict = {}
fileContents = f.read()
list_of_disease = []
fileContents = fileContents.split('\n')


fileContents.pop(34)

for i in range(1,len(fileContents)-1):
    fileContents[i] = fileContents[i].split(',')
    naming_dict["DWI"+str(i)] = fileContents[i][5]
    list_of_disease.append(fileContents[i][5])
naming_dict['DWI33'] = 'Lacunar infact in posterior limb of left internal capsule'
list_of_disease[32] = 'Lacunar infact in posterior limb of left internal capsule'
# print(naming_dict)

naming_dict['DWI3'] = 'Right parietal lobe'
naming_dict['DWI12'] = 'Bilateral cerebellar hemispheres'
naming_dict['DWI16'] = 'Left parietal lobe'
naming_dict['DWI35'] = 'Right corona radiata'
naming_dict['DWI40'] = 'Bilateral occipital lobes'
naming_dict['DWI44'] = 'Right parietal lobe'

list_of_disease[2] = 'Right parietal lobe'
list_of_disease[11] ='Bilateral cerebellar hemispheres'
list_of_disease[15] = 'Left parietal lobe'
list_of_disease[32] = 'Lacunar infarct in posterior limb of left internal capsule'
list_of_disease[34] =  'Right corona radiata'
list_of_disease[39] = 'Bilateral occipital lobes'
list_of_disease[43] = 'Right parietal lobe'

list_of_dict = list(v for v in naming_dict.values())


In [2]:
import shutil
import os

try:
    os.mkdir('C:/Users/Dell/Desktop/DL/TRAIN_DATA')
    os.mkdir('C:/Users/Dell/Desktop/DL/TEST_DATA')
except:
    pass
src = 'C:/Users/Dell/Desktop/DL/CLEANED_DATA/'
dst = 'C:/Users/Dell/Desktop/DL/TRAIN_DATA/'
dst2 = 'C:/Users/Dell/Desktop/DL/TEST_DATA/'
ckeck = False
list_unique = list(set(list_of_disease))
for i in list_unique:
    check = True
    for j in range (1,51):
        try:
            if(check):
                shutil.copy(src = src+str(i)+"/DWI"+str(j)+".jpg",dst=dst+"DWI"+str(j)+".jpg")
                check = False  
            else :
                shutil.copy(src = src+str(i)+"/DWI"+str(j)+".jpg",dst=dst2+"DWI"+str(j)+".jpg")
        except:
            pass
    
                                
        

list_of_dir = []
for x in os.listdir(src):
    list_of_dir.append(x)
    


In [3]:
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Activation, Dropout

In [4]:
path = "C:/Users/Dell/Desktop/DL/TRAIN_DATA"
training_data = []
i =1
size = 100
for img in os.listdir(path):
    try:
        x=img.replace(".jpg","")
        image = cv2.imread(os.path.join(path,img),cv2.IMREAD_GRAYSCALE)
        image = cv2.resize(image,(size,size))
        training_data.append([image,naming_dict[x]])
        i=i+1
    except:
        pass


path = "C:/Users/Dell/Desktop/DL/TEST_DATA"
test_data = []
i =1


for img in os.listdir(path):
    try:
        x=img.replace(".jpg","")
        image = cv2.imread(os.path.join(path,img),cv2.IMREAD_GRAYSCALE)
        image = cv2.resize(image,(size,size))
        test_data.append([image,naming_dict[x]])
        i=i+1
    except:
        pass


In [5]:
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import ImageDataGenerator
from numpy import expand_dims

X = []
Y = []

for images,label in training_data:
    X.append(images)
    Y.append(label)
    
X_test = []
Y_test = []

for images,label in test_data:
    X_test.append(images)
    Y_test.append(label) 

 

Using TensorFlow backend.


In [6]:
X_aug = []
Y_aug = []
X_aug_test = []
Y_aug_test = []


#Data augmentation function with paratmeters
datagen = ImageDataGenerator( width_shift_range=0.05, 
                                 height_shift_range=0.05,
                                 zoom_range=0.1,
                                 shear_range=0.1,
                                 rescale=1./255)

#Data Augmentation Implementation
for images,label in training_data:
    data = img_to_array(images)
    samples = expand_dims(data,0)
    it = datagen.flow(samples, batch_size=1,save_to_dir="./aug_images", save_prefix=label, save_format="jpg")
    for i in range(10):
        temp = it.next()
        X_aug.append(temp)
        Y_aug.append(label)

print(np.shape(X_aug))
print(np.shape(Y_aug))

(460, 1, 100, 100, 1)
(460,)


In [7]:
npX = np.array(X).reshape(-1,size,size,1)
npY = np.array(Y).reshape(-1)
npX_aug = np.array(X_aug).reshape(-1,size,size,1)
npY_aug = np.array(Y_aug).reshape(-1)

print(np.shape(npX))
print(np.shape(npY)) 
print(np.shape(npX_aug))
print(np.shape(npY_aug)) 

npX_test = np.array(X_test).reshape(-1,size,size,1)
npY_test = np.array(Y_test).reshape(-1)
npX_aug_test = np.array(X_aug_test).reshape(-1,size,size,1)
npY_aug_test = np.array(Y_aug_test).reshape(-1)

print(np.shape(npX_test))
print(np.shape(npY_test))

(46, 100, 100, 1)
(46,)
(460, 100, 100, 1)
(460,)
(4, 100, 100, 1)
(4,)


In [13]:
model = Sequential()
model.add(Conv2D((64),(3,3), input_shape = (size,size,1), activation = 'relu' ))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D((128),(3,3), activation = 'relu' ))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
# model.add(Conv2D((256),(3,3), activation = 'relu' ))
# model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
# model.add(Dense(128 , activation = 'relu' ))
# model.add(Dense(64 , activation = 'relu' ))
model.add(Dense(40 , activation = 'softmax' ))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 98, 98, 64)        640       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 49, 49, 64)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 49, 49, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 47, 47, 128)       73856     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 23, 23, 128)       0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 23, 23, 128)       0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 67712)            

In [14]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

encoder = LabelEncoder()

transformed_label = encoder.fit_transform(npY)
transformed_label2 = encoder.fit_transform(npY_aug)
print("Transformed label is :",transformed_label)

y_test_binary = []
y_binary = to_categorical(transformed_label)
y_binary2 = to_categorical(transformed_label2)
for i in Y_test:
    y_test_binary.append(y_binary[np.where(npY==i)[0][0]])
    
y_test_binary = np.array(y_test_binary)

print(y_binary2.shape)  
print(y_test_binary.shape)

x_train, x_validate, y_train, y_validate = train_test_split(npX_aug,y_binary2,test_size=0.1,shuffle=True)


Transformed label is : [39 36 28  0 31 35 13 20 14 38  3 24  8 16 33 17  2 19 12 27  9 36  5  4
  0 28 18 25  6  7 10  2  1 23 34 36 11 30 21 15 26 22 29 37 32 20]
(460, 40)
(4, 40)


In [15]:
model.compile(optimizer = "adam", loss = 'categorical_crossentropy',metrics = ['accuracy'])
model.fit(x_train, y_train ,validation_data=(x_validate,y_validate), batch_size = 1, epochs = 10, shuffle = True)

Train on 414 samples, validate on 46 samples
Epoch 1/10
414/414 [==============================] - 22s 54ms/sample - loss: 3.5585 - acc: 0.0459 - val_loss: 3.1601 - val_acc: 0.1304
Epoch 2/10
414/414 [==============================] - 22s 53ms/sample - loss: 2.9495 - acc: 0.2053 - val_loss: 2.4254 - val_acc: 0.3261
Epoch 3/10
414/414 [==============================] - 21s 51ms/sample - loss: 1.5326 - acc: 0.6039 - val_loss: 1.3251 - val_acc: 0.6522
Epoch 4/10
414/414 [==============================] - 21s 51ms/sample - loss: 0.4125 - acc: 0.8913 - val_loss: 0.8012 - val_acc: 0.7174
Epoch 5/10
414/414 [==============================] - 22s 52ms/sample - loss: 0.0749 - acc: 0.9758 - val_loss: 0.6015 - val_acc: 0.8043
Epoch 6/10
414/414 [==============================] - 21s 52ms/sample - loss: 0.0169 - acc: 0.9952 - val_loss: 0.4689 - val_acc: 0.8478
Epoch 7/10
414/414 [==============================] - 22s 53ms/sample - loss: 0.0160 - acc: 0.9976 - val_loss: 0.5151 - val_acc: 0.8478
Epo

In [16]:
predictions = model.predict(npX)
print(predictions)
acc = 0
for i in range(46):
    ans = np.argmax(predictions[i])
    print("Prediction of ",Y[i]," is ",list_of_dict[int(np.where(transformed_label==ans)[0][0])+1])
    if(ans==transformed_label[i]):
        acc=acc+1
per_acc = (acc/46)*100
print("Accuracy = ",per_acc)


[[0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Prediction of  Splenium of the corpus callosum  is  Pontine infarct on the right
Prediction of  Right parietal lobe  is  Right parietal lobe
Prediction of  Right corona radiata  is  Left centrum semi ovale and right parietal lobe
Prediction of  Bilateral cerebellar hemispheres  is  Right cerebellar hemisphere
Prediction of  Right fronto-parieto-temporo- occipital lobes  is  Right frontal lobe
Prediction of  Right occipital lobe  is  Right temporal lobe
Prediction of  Left frontal lobe  is  Right ganglio-capsular region
Prediction of  Left parietal lobe  is  Left parietal lobe
Prediction of  Left frontal lobe in precentral gyral location  is  Right corona radiata
Prediction of  Right thalamus  is  Right corona radiata
Prediction of  Brainstem  is  Bilateral cerebellar hemispheres
Prediction of  Pontine infarct on the right  is  Right

In [22]:
predictions = model.predict(npX_test)
print(predictions)
acc = 0
for i in range(4):
    ans = np.argmax(predictions[i])

    print(Y_test[i],"(is matched with)",list_of_disease[int(np.where(transformed_label==ans)[0][0])-1])
    if(ans==transformed_label[i]):
        acc=acc+1
per_acc = (acc/4)*100
print("Accuracy = ",per_acc)

[[0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  1.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00]
 [0.0000000e+00 0.0000000e+00 1.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.000

In [18]:
model_fname = "./model/try.h5"
model.save(model_fname)


In [19]:
import tensorflow as tf
from tensorflow.python.framework import graph_io
from tensorflow.keras.models import load_model


# Clear any previous session.
tf.keras.backend.clear_session()

save_pb_dir = './model'
model_fname = './model/try.h5'
def freeze_graph(graph, session, output, save_pb_dir='.', save_pb_name='frozen_model.pb', save_pb_as_text=False):
    with graph.as_default():
        graphdef_inf = tf.graph_util.remove_training_nodes(graph.as_graph_def())
        graphdef_frozen = tf.graph_util.convert_variables_to_constants(session, graphdef_inf, output)
        graph_io.write_graph(graphdef_frozen, save_pb_dir, save_pb_name, as_text=save_pb_as_text)
        return graphdef_frozen

# This line must be executed before loading Keras model.
tf.keras.backend.set_learning_phase(0) 

model = load_model(model_fname)

session = tf.keras.backend.get_session()

INPUT_NODE = [t.op.name for t in model.inputs]
OUTPUT_NODE = [t.op.name for t in model.outputs]
print(INPUT_NODE, OUTPUT_NODE)
frozen_graph = freeze_graph(session.graph, session, [out.op.name for out in model.outputs], save_pb_dir=save_pb_dir)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
['conv2d_2_input'] ['dense_1/Softmax']
Instructions for updating:
Use `tf.compat.v1.graph_util.remove_training_nodes`
Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
INFO:tensorflow:Froze 6 variables.
INFO:tensorflow:Converted 6 variables to const ops.


In [20]:
import platform
is_win = 'windows' in platform.platform().lower()

# OpenVINO 2019
if is_win:
    mo_tf_path = '"C:\Program Files (x86)\IntelSWTools\openvino\deployment_tools\model_optimizer\mo_tf.py"'
else:
    # mo_tf.py path in Linux
    mo_tf_path = '/opt/intel/openvino/deployment_tools/model_optimizer/mo_tf.py'

pb_file = './model/frozen_model.pb'
output_dir = './model'
img_height = 100
input_shape = [1,img_height,img_height,1]
input_shape_str = str(input_shape).replace(' ','')
input_shape_str
# print(input_shape_str)

!python {mo_tf_path} --input_model {pb_file} --output_dir {output_dir} --input_shape {input_shape_str} --data_type FP32

Model Optimizer arguments:
Common parameters:
	- Path to the Input Model: 	C:\Users\Dell\Desktop\DL\./model/frozen_model.pb
	- Path for generated IR: 	C:\Users\Dell\Desktop\DL\./model
	- IR output name: 	frozen_model
	- Log level: 	ERROR
	- Batch: 	Not specified, inherited from the model
	- Input layers: 	Not specified, inherited from the model
	- Output layers: 	Not specified, inherited from the model
	- Input shapes: 	[1,100,100,1]
	- Mean values: 	Not specified
	- Scale values: 	Not specified
	- Scale factor: 	Not specified
	- Precision of IR: 	FP32
	- Enable fusing: 	True
	- Enable grouped convolutions fusing: 	True
	- Move mean values to preprocess section: 	False
	- Reverse input channels: 	False
TensorFlow specific parameters:
	- Input model in text protobuf format: 	False
	- Path to model dump for TensorBoard: 	None
	- List of shared libraries with TensorFlow custom layers implementation: 	None
	- Update the configuration file with input/output node names: 	None
	- Use configur

2020-05-01 10:38:57.178376: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'cudart64_100.dll'; dlerror: cudart64_100.dll not found
2020-05-01 10:38:57.178625: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
